# API 및 크롤링

In [1]:
#!pip list
!pip install squarify

## 환율정보

In [10]:
import requests as req
from tqdm import tqdm #시각화
import time as t
url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON?authkey=ciy2m90S2IZQnhhGYX3vfFcsqe0ezmxV&data=AP01'
gdata = req.get(url).json()
for v in tqdm(gdata):
    if '미국' in v['cur_nm']:
        dwon = v['deal_bas_r']
        print(f'미국의 원달러 환율은 {dwon}입니다.')
    t.sleep(0.5) #테스트용 지연

 96%|█████████▌| 22/23 [00:11<00:00,  1.97it/s]

미국의 원달러 환율은 1,433.2입니다.


100%|██████████| 23/23 [00:11<00:00,  1.97it/s]


## 비트코인 정보

In [16]:
import requests as req
url = 'https://api.bithumb.com/public/ticker/ALL_KRW'
gdata = req.get(url).json()
btc = gdata['data']['BTC']
maxp = float(btc['max_price']) #최대가
minp = float(btc['min_price']) #최소가
openp = float(btc['opening_price']) #시작가
bandp = float(maxp) - float(minp) # 변화폭
print(f'최대가:{maxp}, 최소가:{minp}, 시작가:{openp}, 변화폭:{bandp}')
if (openp + bandp) > maxp:
    print('상승장입니다.')
else:
    print('하강장입니다.')

최대가:141077000.0, 최소가:136000000.0, 시작가:140729000.0, 변화폭:5077000.0
상승장입니다.


## 미세먼지정보


In [20]:
#!pip install --upgrade pip
!pip install python-dotenv

In [2]:
import requests as req # requests 라이브러리 임포트
import os # OS 관련 작업을 위한 라이브러리
from dotenv import load_dotenv # 환경 변수를 로드하기 위한 라이브러리
load_dotenv() # .env 파일에서 환경 변수 로드
key_ori = os.getenv('SER_KEY') # SER_KEY 환경 변수에서 API 키 불러오기
key = req.utils.unquote(key_ori) # URL 인코딩된 키 디코딩
#print(key)
url = 'http://apis.data.go.kr/6260000/AirQualityInfoService/getAirQualityInfoClassifiedByStation' # OpenAPI URL 설정
params = {'serviceKey':key,'pageNo':'1','numOfRows':200,'resultType':'json'} # 요청할 파라미터 설정
response = req.get(url,params).json() # API에 GET 요청을 보내고 JSON 형태로 응답을 받기
data = response['response']['body']['items']['item'] # API 응답에서 'item' 데이터를 'data' 변수에 저장
inp = input('원하는 지역이름을 적어주세요') or '전포동' # 사용자로부터 지역 이름을 입력 받음. 기본값은 '전포동'
if inp not in [s['site'] for s in data]: # 입력한 지역 이름이 'data'에 포함되어 있는지 확인
    print('해당 지역 정보가 없습니다.') # 지역 정보가 없다면 출력
else: 
    for v in data: # 입력한 지역에 해당하는 미세먼지 정보를 출력
        if(v['site']==inp):  # 지역, 미세먼지, 초미세먼지 정보 출력
            print(f"=={v['controlnumber']}==\n{inp} 지역\n미세먼지: {v['pm10']}\n초미세먼지: {v['pm25']}\n{'=' * 14}")

원하는 지역이름을 적어주세요 광안동


==2024121009==
광안동 지역
미세먼지: 18.0
초미세먼지: 17.0
==2024121008==
광안동 지역
미세먼지: 23.0
초미세먼지: 16.0
==2024121007==
광안동 지역
미세먼지: 21.0
초미세먼지: 19.0
==2024121006==
광안동 지역
미세먼지: 19.0
초미세먼지: 20.0
==2024121005==
광안동 지역
미세먼지: 21.0
초미세먼지: 23.0
==2024121004==
광안동 지역
미세먼지: 24.0
초미세먼지: 10.0


## 네이버API 검색(뉴스) 

In [ ]:
# 네이버 검색 API 예제 - 블로그 검색
import os
import sys
import urllib.request
import json
from dotenv import load_dotenv
load_dotenv()
client_id = os.getenv("YOUR_CLIENT_ID") 
#key = req.utils.unquote(client_id)
client_secret = os.getenv("YOUR_CLIENT_SECRET") 
#key = req.utils.unquote(client_secret) 


encText = urllib.parse.quote("속보") #인코딩 중 
url = "https://openapi.naver.com/v1/search/blog?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    data = response_body.decode('utf-8')
    jdata = json.loads(data)
    result = jdata['items']
    for r in result:
        print('*' * 90)
        print('제목:',r['title'])
        print('LinK:',r['link'])
        print('내용:',r['description'])
        print()
        
else:
    print("Error Code:" + rescode)

## 클롤링 네이버 주식 정보 

In [39]:
import requests as req
from bs4 import BeautifulSoup as bs

url = 'https://finance.naver.com/sise/sise_market_sum.naver'
web = req.get(url)

#html기본
#html = web.text
#print(html.find('삼성'))
#print(html[35073:35077])

# soup사용
soup = bs(web.content,'html.parser')
rows = soup.find_all('tr')
#contents = soup.find_all(class_="type_2")
#contents = soup.find_all(class_="tltle")
#contents = soup.find_all(class_="number")
for i, row in enumerate(rows,1):
    title = row.find(class_="tltle")
    number = row.find(class_="number")
    if title and number:
        print(f'{i}) {title.text} - 현재가: {number.text}')
#print(contents)

8) 삼성전자 - 현재가: 54,000
9) SK하이닉스 - 현재가: 169,900
10) LG에너지솔루션 - 현재가: 378,500
11) 삼성바이오로직스 - 현재가: 957,000
12) 현대차 - 현재가: 208,500
16) 셀트리온 - 현재가: 184,100
17) 기아 - 현재가: 96,500
18) 삼성전자우 - 현재가: 45,650
19) NAVER - 현재가: 215,000
20) KB금융 - 현재가: 84,800
24) 고려아연 - 현재가: 1,436,000
25) 신한지주 - 현재가: 50,400
26) 현대모비스 - 현재가: 248,000
27) POSCO홀딩스 - 현재가: 270,500
28) 삼성물산 - 현재가: 115,700
32) HD현대중공업 - 현재가: 230,000
33) 삼성생명 - 현재가: 101,600
34) 카카오 - 현재가: 44,350
35) 메리츠금융지주 - 현재가: 101,800
36) LG화학 - 현재가: 262,000
40) SK이노베이션 - 현재가: 117,600
41) 삼성화재 - 현재가: 372,000
42) 삼성SDI - 현재가: 248,500
43) 하나금융지주 - 현재가: 58,900
44) HMM - 현재가: 18,770
48) 크래프톤 - 현재가: 314,500
49) HD한국조선해양 - 현재가: 211,500
50) KT&G - 현재가: 116,500
51) LG전자 - 현재가: 85,300
52) 한국전력 - 현재가: 21,400
56) 한화에어로스페이스 - 현재가: 299,500
57) HD현대일렉트릭 - 현재가: 363,500
58) 포스코퓨처엠 - 현재가: 165,500
59) SK텔레콤 - 현재가: 56,400
60) LG - 현재가: 75,500
64) 기업은행 - 현재가: 14,650
65) 우리금융지주 - 현재가: 15,520
66) 카카오뱅크 - 현재가: 23,950
67) KT - 현재가: 44,400
68) 두산에너빌리티 - 현재가: 17,390
72) 삼성에스디에스 - 현

## 크롤링 멜론 순위

In [6]:
import requests as req
from bs4 import BeautifulSoup as bs

# 올바른 headers 딕셔너리 정의
url = 'https://www.melon.com/chart/index.htm'
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.1.1 Safari/605.1.15"}
web = req.get(url, headers=headers)
# BeautifulSoup을 이용해 HTML 파싱
soup = bs(web.content, 'html.parser')
day = soup.select_one('.year')
hour = soup.select_one('.hour')
# 노래 정보 추출
rank = soup.select('.wrap_song_info span')
title = soup.select('.wrap_song_info .rank01')
artist= soup.select('.wrap_song_info .checkEllipsis')
print(day.text, hour.text)
for i, (t , a) in enumerate(zip(title, artist),1):
    print(i,'위', t.text.strip(), '/', a.text.strip())

2024.12.12 09:00
1 위 HOME SWEET HOME (feat. 태양, 대성) / G-DRAGON
2 위 APT. / 로제 (ROSÉ), Bruno Mars
3 위 Whiplash / aespa
4 위 나는 반딧불 / 황가람
5 위 POWER / G-DRAGON
6 위 toxic till the end / 로제 (ROSÉ)
7 위 HAPPY / DAY6 (데이식스)
8 위 첫 눈 / EXO
9 위 UP (KARINA Solo) / aespa
10 위 내 이름 맑음 / QWER
11 위 내게 사랑이 뭐냐고 물어본다면 / 로이킴
12 위 너와의 모든 지금 / 재쓰비 (JAESSBEE)
13 위 Welcome to the Show / DAY6 (데이식스)
14 위 Mantra / 제니 (JENNIE)
15 위 Supernova / aespa
16 위 소나기 / 이클립스 (ECLIPSE)
17 위 한 페이지가 될 수 있게 / DAY6 (데이식스)
18 위 Drowning / WOODZ
19 위 사랑은 늘 도망가 / 임영웅
20 위 천상연 / 이창섭
21 위 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지 / AKMU (악뮤)
22 위 슬픈 초대장 / 순순희(지환)
23 위 number one girl / 로제 (ROSÉ)
24 위 고민중독 / QWER
25 위 Pump Up The Volume! / PLAVE
26 위 예뻤어 / DAY6 (데이식스)
27 위 WAY 4 LUV / PLAVE
28 위 우리들의 블루스 / 임영웅
29 위 Love wins all / 아이유
30 위 온기 / 임영웅
31 위 Supersonic / 프로미스나인
32 위 Home / 임영웅
33 위 How Sweet / NewJeans
34 위 모래 알갱이 / 임영웅
35 위 All I Want for Christmas Is You / Mariah Carey
36 위 다시 만날 수 있을까 / 임영웅
37 위 첫 만남은 계획대로 되지 않아 / TWS (투어스)
38 위 Small 

## 크롤링 벅스 순위

In [2]:
import requests as req
from bs4 import BeautifulSoup as bs
from datetime import datetime

# 현재 시간 가져오기
now = datetime.now()
formatted_time = now.strftime("%Y.%m.%d %H:%M")

# 벅스 차트 URL과 헤더
url = 'https://music.bugs.co.kr/chart?wl_ref=M_left_02_01'
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.1.1 Safari/605.1.15"}

# 요청 보내기
web = req.get(url, headers=headers)

# BeautifulSoup으로 HTML 파싱
soup = bs(web.content, 'html.parser')

# 곡명과 아티스트 정보 추출
titles = soup.select('p.title a')
artists = soup.select('p.artist a')

# 결과 출력
print(formatted_time)
for i in range(30):  # 30위까지만 출력
    title = titles[i].text.strip()
    artist = artists[i].text.strip()
    print(f"{i + 1}위 {title} - {artist}")

2024.12.12 00:26
1위 toxic till the end - 로제(ROSÉ)
2위 HOME SWEET HOME (feat. 태양, 대성) - G-DRAGON
3위 Whiplash - aespa
4위 APT. - 로제(ROSÉ)
5위 너와의 모든 지금 - 로제(ROSÉ)
6위 number one girl - 재쓰비 (JAESSBEE)
7위 POWER - 로제(ROSÉ)
8위 Drowning - G-DRAGON
9위 나는 반딧불 - WOODZ
10위 Mantra - 황가람
11위 HAPPY - 제니 (JENNIE)
12위 Letter To Myself - DAY6 (데이식스)
13위 UP (KARINA Solo) - 태연 (TAEYEON)
14위 Supernova - aespa
15위 내 이름 맑음 - aespa
16위 Igloo - QWER
17위 사랑을 억지로 할 수 없잖아 - KISS OF LIFE
18위 Welcome to the Show - 황치열
19위 DRIP - DAY6 (데이식스)
20위 첫 눈 - BABYMONSTER
21위 내게 사랑이 뭐냐고 물어본다면 - EXO
22위 한 페이지가 될 수 있게 - 로이킴
23위 고민중독 - DAY6 (데이식스)
24위 How Sweet - QWER
25위 소나기 - NewJeans
26위 All I Want for Christmas Is You - 이클립스 (ECLIPSE)
27위 Supernatural - Mariah Carey(머라이어 캐리)
28위 녹아내려요 - NewJeans
29위 drinks or coffee - DAY6 (데이식스)
30위 Armageddon - 로제(ROSÉ)


## 부산대 학식 정보

In [16]:
!pip install html5lib

In [7]:
import requests as req
from bs4 import BeautifulSoup as bs
url = 'https://pusan.ac.kr/kor/CMS/MenuMgr/menuListOnBuilding.do'
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.1.1 Safari/605.1.15"}
web = req.get(url, headers=headers)
#ㅁsoup = bs(web.content, 'html.parser')
soup = bs(web.content, 'html5lib')
date = soup.select('.menu-tbl .date') 
day = soup.select('.menu-tbl .day') 
won = soup.select('.menu-tbl .menu-tit01')
menu = soup.select('h3.menu-tit01+p')
#print(date, day, won, menu)
for d, y, w, m in zip(date, day, won, menu):
    print('='*20)
    print(d.text, y.text)
    print('='*20)
    print(w.text.rstrip())
    print(m.text)
    print()

2024.12.09 월
	 정식-5,500원
백미밥
물만두국
꿔바로우/소스
순대매콤볶음
연두부/부추양념장
상추겉절이
포기김치


2024.12.10 화
	  정식-5,500원
잡곡밥
감자탕
치킨치즈가스/소스
우동굴소스볶음
후랑크햄옥수수콘볶음
열무겉절이
포기김치


2024.12.11 수
	 정식-5,500원
강황밥
모듬어묵국
돈육김치찜
만두/비빔야채
김가루견과무침
콩나물무침
포기김치

2024.12.12 목
	 정식-5,500원
백미밥
버섯된장국
떡갈비/소스
카레
실곤약맛살샐러드
청경채치커리겉절이
포기김치


2024.12.13 금
정식-5,500원
잡곡밥
국수
순살치킨/맛초킹소스
사각어묵매콤무침
양상추야채샐러드/소스
상추겉절이
포기김치




In [3]:
import test01

In [5]:
test01.hihi()

반갑습니다.


## 부산외대 학식 정보

In [8]:
!pip install certifi

In [ ]:
import requests as req
import certifi
from bs4 import BeautifulSoup as bs
url = 'https://www.bufs.ac.kr/bbs/my/ajax.view.skin.php?bo_table=weekly_meal&wr_id=1'
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.1.1 Safari/605.1.15"}

web = req.get(url, headers=headers, verify=certifi.where())
soup = bs(web.content, 'html.parser')
date = soup.select('table th')  # 간단화된 예
menu = soup.select('table p')  # 간단화된 예

for d, m in zip(date, menu):
    print(date)
    print(menu)

In [1]:
import foodlist as fd

In [2]:
fd.pr()

2024.12.09 월
	 정식-5,500원
백미밥
물만두국
꿔바로우/소스
순대매콤볶음
연두부/부추양념장
상추겉절이
포기김치


2024.12.10 화
	  정식-5,500원
잡곡밥
감자탕
치킨치즈가스/소스
우동굴소스볶음
후랑크햄옥수수콘볶음
열무겉절이
포기김치


2024.12.11 수
	 정식-5,500원
강황밥
모듬어묵국
돈육김치찜
만두/비빔야채
김가루견과무침
콩나물무침
포기김치

2024.12.12 목
	 정식-5,500원
백미밥
버섯된장국
떡갈비/소스
카레
실곤약맛살샐러드
청경채치커리겉절이
포기김치


2024.12.13 금
정식-5,500원
잡곡밥
국수
순살치킨/맛초킹소스
사각어묵매콤무침
양상추야채샐러드/소스
상추겉절이
포기김치


